To Do:
 - [X] join in NCSL
 - [X] join local opposition
 - [X] model state level ordinances
 - [ ] make resource class
 - [X] is_hybrid
 - [X] has_opposition
 - [ ] replace "Unknown" ISO counties with NULL before geocoding. It fills in garbage otherwise 

In [53]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
%autoreload 2

In [55]:
import dbcp
import pandas as pd

In [56]:
from dbcp.data_mart import projects as pj

In [57]:
engine = dbcp.helpers.get_sql_engine()

In [58]:
loc_df = pj._get_iso_location_df(engine)

In [59]:
# no multi-valued projects
assert loc_df['project_id'].nunique() == loc_df.shape[0]

In [60]:
res_df = pj._get_iso_resource_df(engine)

In [61]:
res_df.head()

,capacity_mw,project_id,resource_clean
0,500.0,0,Battery Storage
1,500.0,1,Battery Storage
2,725.0,3,Battery Storage
3,4.5,4,Onshore Wind
4,80.0,5,Battery Storage


In [62]:
res_df['resource_clean'].value_counts()

Solar                                 6294
Onshore Wind                          2512
Natural Gas                           1997
Battery Storage                       1964
Biomass                                265
Unknown                                250
Other Storage                          232
Coal                                   215
Hydro                                  185
Nuclear                                 98
Oil                                     97
Offshore Wind                           95
Other                                   58
Solar; Storage                          58
Geothermal                              49
Combustion Turbine                      39
Biofuel                                 21
Landfill Gas                            19
Steam                                   12
Waste Heat                              10
Pumped Storage                          10
Fuel Cell                                8
Wind; Storage                            6
Municipal S

In [63]:
unparsed_hybrids = res_df.loc[res_df['resource_clean'].str.contains(';'),:]
unparsed_hybrids.shape

(69, 3)

In [64]:
is_unparsed_hybrid = res_df.loc[res_df.loc[:,'project_id'].isin(pd.Index(unparsed_hybrids['project_id']))]
is_unparsed_hybrid.head()

,capacity_mw,project_id,resource_clean
7738,1.100000,7316,Solar; Storage
9472,270.459992,8980,Wind; Storage
9475,241.490005,8983,Wind; Storage
9487,80.000000,8995,Oil; Biomass
9493,20.000000,9002,Solar; Storage


In [65]:
is_unparsed_hybrid.shape

(69, 3)

In [66]:
from re import IGNORECASE

In [67]:
res_df.loc[res_df['resource_clean'].str.contains('storage', flags=IGNORECASE),:]['project_id'].shape

(2272,)

In [68]:
res_df.groupby('project_id').size().value_counts()

1    12475
2      998
3       10
dtype: int64

In [69]:
is_storage = res_df.loc[:,'resource_clean'].str.contains('storage', flags=IGNORECASE)

In [70]:
res_df['storage_type'] = res_df.loc[:,'resource_clean'].where(is_storage)
res_df['generation_type'] = res_df.loc[:,'resource_clean'].where(~is_storage)

In [71]:
res_df.head(10)

,capacity_mw,project_id,resource_clean,storage_type,generation_type
0,500.0,0,Battery Storage,Battery Storage,NaN
1,500.0,1,Battery Storage,Battery Storage,NaN
2,725.0,3,Battery Storage,Battery Storage,NaN
3,4.5,4,Onshore Wind,NaN,Onshore Wind
4,80.0,5,Battery Storage,Battery Storage,NaN
5,800.0,6,Solar,NaN,Solar
6,75.0,7,Solar,NaN,Solar
7,250.0,8,Onshore Wind,NaN,Onshore Wind
8,99.0,9,Solar,NaN,Solar
9,NaN,10,Battery Storage,Battery Storage,NaN


In [72]:
res_df.groupby('project_id').nth(2).shape

(10, 4)

In [73]:
gen = res_df.loc[~is_storage,:]
storage = res_df.loc[is_storage,:]

In [74]:
gen.shape, storage.shape

((12229, 5), (2272, 5))

In [75]:
out = gen.groupby('project_id')[['generation_type', 'capacity_mw']].nth(0).rename(columns={'generation_type': 'generation_type_1', 'capacity_mw': 'generation_capacity_mw_1'})
out.head(10)

,generation_type_1,generation_capacity_mw_1
project_id,,
4,Onshore Wind,4.5
6,Solar,800.0
7,Solar,75.0
8,Onshore Wind,250.0
9,Solar,99.0
10,Solar,400.0
11,Solar,243.0
13,Solar,270.0
14,Solar,83.0


In [76]:
two_gens = gen.groupby('project_id')[['generation_type', 'capacity_mw']].nth(1).rename(columns={'generation_type': 'generation_type_2', 'capacity_mw': 'generation_capacity_mw_2'})

In [77]:
assert gen.groupby('project_id')[['generation_type', 'capacity_mw']].nth(2).shape[0] == 0

In [78]:
out = out.join(two_gens, how='left')

In [79]:
assert storage.shape[0] == storage.groupby('project_id').ngroups

In [80]:
storage = storage.set_index('project_id', verify_integrity=True)[['storage_type', 'capacity_mw']].rename(columns={'capacity_mw': 'storage_capacity_mw'})
storage.head(10)

,storage_type,storage_capacity_mw
project_id,,
0,Battery Storage,500.0
1,Battery Storage,500.0
3,Battery Storage,725.0
5,Battery Storage,80.0
10,Battery Storage,NaN
11,Battery Storage,91.0
12,Battery Storage,30.0
13,Battery Storage,270.0
15,Battery Storage,NaN


In [81]:
out = out.join(storage, how='outer')

In [82]:
assert out.shape[0] == res_df['project_id'].nunique()

In [83]:
test = pj._convert_resource_df_long_to_wide(res_df)

In [84]:
pd.testing.assert_frame_equal(out.sort_index(), test)

In [85]:
test = pj._get_and_join_iso_tables()

In [86]:
test.head()

,project_id,generation_type_1,generation_capacity_mw_1,generation_type_2,generation_capacity_mw_2,storage_type,storage_capacity_mw,date_operational,date_proposed,date_withdrawn,...,interconnection_status_lbnl,point_of_interconnection,project_name,queue_date,queue_status,region,utility,withdrawl_reason,state_id_fips,county_id_fips
0,0,NaN,NaN,NaN,NaN,Battery Storage,500.0,NaT,2023-12-31,NaT,...,In Progress,500kV at Broadview Substation,None,2020-08-13,active,West (non-ISO),Colstrip,None,30,30111
1,1,NaN,NaN,NaN,NaN,Battery Storage,500.0,NaT,2023-12-31,NaT,...,In Progress,Broadview substation,None,2020-07-17,active,West (non-ISO),Colstrip,None,30,30111
2,3,NaN,NaN,NaN,NaN,Battery Storage,725.0,NaT,2024-11-15,NaT,...,Not Started,Valley Substation 500kV,MENIFEE POWER BANK,2019-04-15,active,CAISO,None,None,06,06065
3,4,Onshore Wind,4.5,NaN,NaN,NaN,NaN,NaT,2021-12-31,NaT,...,IA Executed,Breckinridge 138kV,None,NaT,active,SPP,OKGE,None,40,40047
4,5,NaN,NaN,NaN,NaN,Battery Storage,80.0,NaT,NaT,NaT,...,In Progress,Fort Supply SW 138kV Substation,None,NaT,active,SPP,None,None,40,40153


In [87]:
test.columns

Index(['project_id', 'generation_type_1', 'generation_capacity_mw_1',
       'generation_type_2', 'generation_capacity_mw_2', 'storage_type',
       'storage_capacity_mw', 'date_operational', 'date_proposed',
       'date_withdrawn', 'days_in_queue', 'developer', 'entity',
       'interconnection_status_lbnl', 'point_of_interconnection',
       'project_name', 'queue_date', 'queue_status', 'region', 'utility',
       'withdrawl_reason', 'state_id_fips', 'county_id_fips'],
      dtype='object')

In [88]:
test['generation_type_2'].notna().agg(['sum', 'mean'])

sum     41.00000
mean     0.00304
Name: generation_type_2, dtype: float64

In [89]:
ncsl = pj._get_ncsl_wind_permitting_df(engine)
ncsl.head()

,description,permitting_type,state_id_fips
0,According to the Wind Energy Technology Office...,Local,01
1,A Certificate of Convenience and Necessity iss...,Hybrid,02
2,Utilities planning to construct an energy faci...,Hybrid,04
3,New construction of larger facilities providin...,Local,05
4,"Land use decisions, including wind siting, are...",Local,06


In [90]:
test.head().merge(ncsl, on='state_id_fips', how='left')

,project_id,generation_type_1,generation_capacity_mw_1,generation_type_2,generation_capacity_mw_2,storage_type,storage_capacity_mw,date_operational,date_proposed,date_withdrawn,...,project_name,queue_date,queue_status,region,utility,withdrawl_reason,state_id_fips,county_id_fips,description,permitting_type
0,0,NaN,NaN,NaN,NaN,Battery Storage,500.0,NaT,2023-12-31,NaT,...,None,2020-08-13,active,West (non-ISO),Colstrip,None,30,30111,There is no state level siting authority for w...,Local
1,1,NaN,NaN,NaN,NaN,Battery Storage,500.0,NaT,2023-12-31,NaT,...,None,2020-07-17,active,West (non-ISO),Colstrip,None,30,30111,There is no state level siting authority for w...,Local
2,3,NaN,NaN,NaN,NaN,Battery Storage,725.0,NaT,2024-11-15,NaT,...,MENIFEE POWER BANK,2019-04-15,active,CAISO,None,None,06,06065,"Land use decisions, including wind siting, are...",Local
3,4,Onshore Wind,4.5,NaN,NaN,NaN,NaN,NaT,2021-12-31,NaT,...,None,NaT,active,SPP,OKGE,None,40,40047,"Prior to constructing a wind facility, a proje...",Hybrid
4,5,NaN,NaN,NaN,NaN,Battery Storage,80.0,NaT,NaT,NaT,...,None,NaT,active,SPP,None,None,40,40153,"Prior to constructing a wind facility, a proje...",Hybrid


In [91]:
local_opp = pj._get_local_opposition_df(engine)
local_opp.head()

,county_id_fips,earliest_year_mentioned,locality_name,locality_type,ordinance
0,01049,2019.0,Dekalb County,county,"Under a 2019 ordinance, any wind energy system..."
1,01003,NaN,Baldwin County,county,"Large wind energy conversion systems (WECS), U..."
2,06071,2019.0,San Bernardino County,county,"In 2019, the San Bernardino County Board of Su..."
3,06073,NaN,San Diego County,county,San Diego County limits small wind turbine hei...
4,08121,2020.0,Washington County,county,A temporary moratorium on the county’s process...


In [92]:
state_df = pj._get_state_opposition_df(engine)
state_df

,earliest_year_mentioned,policy,state_id_fips
0,2017,"In 2017, the Legislature enacted Public Act No...",09
1,2004,"In 2004, Kansas Governor Kathleen Sebelius ins...",20
2,2018,Governor LePage signed an executive order in J...,23
3,2020,In 2020 New York enacted the Accelerated Renew...,36
4,2019,"Legislative amendments enacted on May 23, 2019...",41


In [93]:
# drop states that repealed their policies or whose policy was pro-RE not anti-RE
fips_codes_to_drop = {'23', '36'} # Maine, New York
filtered_state_df = state_df.loc[~state_df.loc[:, 'state_id_fips'].isin(fips_codes_to_drop),:]

In [94]:
filtered_state_df

,earliest_year_mentioned,policy,state_id_fips
0,2017,"In 2017, the Legislature enacted Public Act No...",09
1,2004,"In 2004, Kansas Governor Kathleen Sebelius ins...",20
4,2019,"Legislative amendments enacted on May 23, 2019...",41


In [95]:
all_counties = pj._get_county_fips_df(engine)
all_states = pj._get_state_fips_df(engine)

In [96]:
all_counties

,state_id_fips,county_name,county_id_fips
0,01,Autauga County,01001
1,01,Baldwin County,01003
2,01,Barbour County,01005
3,01,Bibb County,01007
4,01,Blount County,01009
...,...,...,...
3231,72,Yauco Municipio,72153
3232,74,Midway Islands District,74300
3233,78,St. Croix Island District,78010
3234,78,St. John Island District,78020


In [97]:
states_as_counties = filtered_state_df.merge(all_counties.loc[:, ['county_id_fips', 'state_id_fips']], on='state_id_fips', how='left')
states_as_counties = states_as_counties.merge(all_states.loc[:, ['state_name', 'state_id_fips']], on='state_id_fips', how='left')
states_as_counties['locality_type'] = 'state'
states_as_counties

,earliest_year_mentioned,policy,state_id_fips,county_id_fips,state_name,locality_type
0,2017,"In 2017, the Legislature enacted Public Act No...",09,09001,Connecticut,state
1,2017,"In 2017, the Legislature enacted Public Act No...",09,09003,Connecticut,state
2,2017,"In 2017, the Legislature enacted Public Act No...",09,09005,Connecticut,state
3,2017,"In 2017, the Legislature enacted Public Act No...",09,09007,Connecticut,state
4,2017,"In 2017, the Legislature enacted Public Act No...",09,09009,Connecticut,state
...,...,...,...,...,...,...
144,2019,"Legislative amendments enacted on May 23, 2019...",41,41063,Oregon,state
145,2019,"Legislative amendments enacted on May 23, 2019...",41,41065,Oregon,state
146,2019,"Legislative amendments enacted on May 23, 2019...",41,41067,Oregon,state
147,2019,"Legislative amendments enacted on May 23, 2019...",41,41069,Oregon,state


In [98]:
combined = states_as_counties.merge(local_opp, on='county_id_fips', how='outer')
combined

,earliest_year_mentioned_x,policy,state_id_fips,county_id_fips,state_name,locality_type_x,earliest_year_mentioned_y,locality_name,locality_type_y,ordinance
0,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09001,Connecticut,state,NaN,NaN,NaN,NaN
1,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09003,Connecticut,state,NaN,NaN,NaN,NaN
2,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09005,Connecticut,state,NaN,NaN,NaN,NaN
3,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09007,Connecticut,state,NaN,NaN,NaN,NaN
4,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09009,Connecticut,state,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
243,NaN,NaN,NaN,49005,NaN,NaN,2017.0,Hyrum,city,A moratorium on new solar power installation w...
244,NaN,NaN,NaN,51023,NaN,NaN,2020.0,Botetourt County,county,A turbine height limit of 550 feet provided by...
245,NaN,NaN,NaN,54037,NaN,NaN,NaN,Jefferson County,county,The Jefferson County Commission is currently c...
246,NaN,NaN,NaN,55071,NaN,NaN,2013.0,Manitowoc County,county,In 2013 the Manitowoc County Board passed a re...


In [99]:
combined['ordinance_earliest_year_mentioned'] = combined['earliest_year_mentioned_y'].fillna(combined['earliest_year_mentioned_x'])

In [100]:
combined['ordinance'].fillna('State Policy: ' + combined['policy'])

0      State Policy: In 2017, the Legislature enacted...
1      State Policy: In 2017, the Legislature enacted...
2      State Policy: In 2017, the Legislature enacted...
3      State Policy: In 2017, the Legislature enacted...
4      State Policy: In 2017, the Legislature enacted...
                             ...                        
243    A moratorium on new solar power installation w...
244    A turbine height limit of 550 feet provided by...
245    The Jefferson County Commission is currently c...
246    In 2013 the Manitowoc County Board passed a re...
247    An ordinance requiring turbine setbacks of a h...
Name: ordinance, Length: 248, dtype: object

In [101]:
has_both = combined[['ordinance', 'policy']].notna().all(axis=1)

In [102]:
# concatenate the intersection
combined.loc[has_both, 'ordinance'] = combined.loc[has_both, 'ordinance'] + (' State Policy: ' + combined.loc[has_both, 'policy'])

In [103]:
combined.head(3)

,earliest_year_mentioned_x,policy,state_id_fips,county_id_fips,state_name,locality_type_x,earliest_year_mentioned_y,locality_name,locality_type_y,ordinance,ordinance_earliest_year_mentioned
0,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09001,Connecticut,state,NaN,NaN,NaN,NaN,2017.0
1,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09003,Connecticut,state,NaN,NaN,NaN,NaN,2017.0
2,2017.0,"In 2017, the Legislature enacted Public Act No...",09,09005,Connecticut,state,NaN,NaN,NaN,NaN,2017.0


In [104]:
local_opp['locality_type'].isna().sum()

0

In [106]:
# have to handle many-to-one ordinance-to-county relationships: 8 counties have multiple ordinances
local_opp.groupby('county_id_fips').size().value_counts()

1    84
2     6
4     1
3     1
dtype: int64

In [107]:
local_opp.loc[local_opp.duplicated(subset='county_id_fips', keep=False),:].sort_values('county_id_fips')

,county_id_fips,earliest_year_mentioned,locality_name,locality_type,ordinance
45,26017,2019.0,Monitor Charter Township,town,"In March 2019, officials unanimously approved ..."
48,26017,2018.0,Beaver Township,town,"Beaver Township enacted a 2,000-foot setback r..."
43,26023,NaN,Branch County,county,The Branch County Concerned Citizens organizat...
44,26023,2020.0,Matteson Township,town,The Planning Commission approved an amendment ...
46,26023,2020.0,Sherwood,city,Wind energy ordinance restrictions passed with...
47,26023,2020.0,Batavia Township,town,"Similarly to Sherwood Township, voters approve..."
63,36063,2020.0,Porter,town,"In January 2020, the town of Porter approved a..."
65,36063,2018.0,Somerset,city,In January 2018 the Somerset town board unanim...
76,40017,1995.0,Yukon,city,A 1995 ordinance sets height limit at 100 feet...
75,40017,2011.0,El Reno,city,A 2011 ordinance sets restrictions on wind tur...


Agg m:1 ordinances to county level:
* take min of earliest year
* if only one locality_name, use it. Otherwise "multiple"
* same with 'locality type'
* concatenate ordinances, each with locality_name prefix 

In [108]:
dupe_counties = local_opp.duplicated(subset='county_id_fips', keep=False)

In [109]:
dupes = local_opp.loc[dupe_counties,:].copy()
not_dupes = local_opp.loc[~dupe_counties,:].copy()

In [110]:
dupes['ordinance'] = dupes['locality_name'] + ': ' + dupes['ordinance'] + '\n'

In [111]:
grp = dupes.groupby('county_id_fips')

In [112]:
# min year
years = grp['earliest_year_mentioned'].min()
years

county_id_fips
26017    2018.0
26023    2020.0
36063    2018.0
40017    1995.0
48061    2020.0
48113    2015.0
48139    2010.0
48439    2020.0
Name: earliest_year_mentioned, dtype: float64

In [113]:
n_unique = grp[['locality_name', 'locality_type']].nunique()
n_unique

,locality_name,locality_type
county_id_fips,,
26017,2,1
26023,4,3
36063,2,2
40017,2,1
48061,1,1
48113,3,1
48139,2,1
48439,1,1


In [114]:
grp[['locality_name', 'locality_type']].nth(0).mask(n_unique > 1, other='multiple')

,locality_name,locality_type
county_id_fips,,
26017,multiple,town
26023,multiple,multiple
36063,multiple,multiple
40017,multiple,city
48061,Brownsville,city
48113,multiple,city
48139,multiple,city
48439,Benbrook,city


In [115]:
grp['ordinance'].sum().str.strip()

county_id_fips
26017    Monitor Charter Township: In March 2019, offic...
26023    Branch County: The Branch County Concerned Cit...
36063    Porter: In January 2020, the town of Porter ap...
40017    El Reno: A 2011 ordinance sets restrictions on...
48061    Brownsville: Enacted July 9, 2020: this ordina...
48113    Grand Prairie: Enacted August 3, 2020, this or...
48139    Midlothian: Enacted in June 2020, a local ordi...
48439    Benbrook: Updated April 29, 2020: the ordinanc...
Name: ordinance, dtype: object

In [117]:
filtered_state_opp = pj._filter_state_opposition(state_df)

In [118]:
states_to_counties = pj._represent_state_opposition_as_counties(filtered_state_opp, county_fips_df=all_counties, state_fips_df=all_states)

In [119]:
states_to_counties

,earliest_year_mentioned,ordinance,county_id_fips,locality_name,locality_type
0,2017,"In 2017, the Legislature enacted Public Act No...",09001,Connecticut,state
1,2017,"In 2017, the Legislature enacted Public Act No...",09003,Connecticut,state
2,2017,"In 2017, the Legislature enacted Public Act No...",09005,Connecticut,state
3,2017,"In 2017, the Legislature enacted Public Act No...",09007,Connecticut,state
4,2017,"In 2017, the Legislature enacted Public Act No...",09009,Connecticut,state
...,...,...,...,...,...
144,2019,"Legislative amendments enacted on May 23, 2019...",41063,Oregon,state
145,2019,"Legislative amendments enacted on May 23, 2019...",41065,Oregon,state
146,2019,"Legislative amendments enacted on May 23, 2019...",41067,Oregon,state
147,2019,"Legislative amendments enacted on May 23, 2019...",41069,Oregon,state


In [120]:
combined_opp = pd.concat([local_opp, states_to_counties], axis=0)
combined_opp = pj._agg_local_ordinances_to_counties(combined_opp)

In [121]:
combined_opp

,county_id_fips,earliest_year_mentioned,locality_name,locality_type,ordinance
1,01003,NaN,Baldwin County,county,"Large wind energy conversion systems (WECS), U..."
0,01049,2019.0,Dekalb County,county,"Under a 2019 ordinance, any wind energy system..."
2,06071,2019.0,San Bernardino County,county,"In 2019, the San Bernardino County Board of Su..."
3,06073,NaN,San Diego County,county,San Diego County limits small wind turbine hei...
4,08121,2020.0,Washington County,county,A temporary moratorium on the county’s process...
...,...,...,...,...,...
75,49005,2017.0,Hyrum,city,A moratorium on new solar power installation w...
76,51023,2020.0,Botetourt County,county,A turbine height limit of 550 feet provided by...
77,54037,NaN,Jefferson County,county,The Jefferson County Commission is currently c...
78,55071,2013.0,Manitowoc County,county,In 2013 the Manitowoc County Board passed a re...


In [122]:
combined_opp[combined_opp['locality_name'] ==  'multiple']

,county_id_fips,earliest_year_mentioned,locality_name,locality_type,ordinance
225,20045,2004.0,multiple,multiple,Douglas County: In response to the NextEra Ene...
226,20107,2004.0,multiple,multiple,Linn County: In response to a wind project pro...
227,20113,2004.0,multiple,multiple,McPherson County: In response to Gamesa Energí...
228,20155,2004.0,multiple,multiple,Reno County: In response to the opposition to ...
229,26017,2018.0,multiple,town,"Monitor Charter Township: In March 2019, offic..."
230,26023,2020.0,multiple,multiple,Branch County: The Branch County Concerned Cit...
231,36063,2018.0,multiple,multiple,"Porter: In January 2020, the town of Porter ap..."
232,40017,1995.0,multiple,city,El Reno: A 2011 ordinance sets restrictions on...
234,48113,2015.0,multiple,city,"Grand Prairie: Enacted August 3, 2020, this or..."
235,48139,2010.0,multiple,city,"Midlothian: Enacted in June 2020, a local ordi..."


In [127]:
out = pj.make_project_data_mart_table()

In [128]:
out

,project_name,project_id,iso_region,entity,utility,developer,state,county,state_id_fips,county_id_fips,...,point_of_interconnection,queue_status,withdrawl_reason,has_ordinance,ordinance_jurisdiction_name,ordinance_jurisdiction_type,ordinance_earliest_year_mentioned,ordinance,state_permitting_type,state_permitting_text
0,None,0,West (non-ISO),NWE,Colstrip,None,MT,Yellowstone County,30,30111,...,500kV at Broadview Substation,active,None,False,NaN,NaN,NaN,NaN,Local,There is no state level siting authority for w...
1,None,1,West (non-ISO),PacifiCorp,Colstrip,None,MT,Yellowstone County,30,30111,...,Broadview substation,active,None,False,NaN,NaN,NaN,NaN,Local,There is no state level siting authority for w...
2,MENIFEE POWER BANK,3,CAISO,CAISO,None,None,CA,Riverside County,06,06065,...,Valley Substation 500kV,active,None,False,NaN,NaN,NaN,NaN,Local,"Land use decisions, including wind siting, are..."
3,None,4,SPP,SPP,OKGE,None,OK,Garfield County,40,40047,...,Breckinridge 138kV,active,None,False,NaN,NaN,NaN,NaN,Hybrid,"Prior to constructing a wind facility, a proje..."
4,None,5,SPP,SPP,None,None,OK,Woodward County,40,40153,...,Fort Supply SW 138kV Substation,active,None,False,NaN,NaN,NaN,NaN,Hybrid,"Prior to constructing a wind facility, a proje..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13483,None,14235,None,PJM,PPL,None,PA,Luzerne County,42,42079,...,Freeland #1 Tap-Jeddo-Highland,withdrawn,None,False,NaN,NaN,NaN,NaN,Local,Local governments have authority to plan and r...
13484,None,14236,None,PJM,ME,None,PA,Adams County,42,42001,...,None,withdrawn,None,False,NaN,NaN,NaN,NaN,Local,Local governments have authority to plan and r...
13485,None,14237,None,PJM,JCPL,None,NJ,Warren County,34,34041,...,Flanders 115 kV,withdrawn,None,False,NaN,NaN,NaN,NaN,Local,Wind generation has an “inherently beneficial ...
13486,None,14239,None,PJM,PPL,None,PA,Montour County,42,42093,...,Derry 12.47 kV,withdrawn,None,False,NaN,NaN,NaN,NaN,Local,Local governments have authority to plan and r...


In [132]:
!pwd

/app/notebooks


In [129]:
out.to_csv('../data/output/project_data_mart.csv', index=False)

In [130]:
out.shape

(13488, 34)

In [131]:
pd.read_csv('../data/output/project_data_mart.csv').shape

(13488, 34)